In [ ]:
from multiinstance.data.realData import buildDataset as buildReal
from multiinstance.utils import *

from multiinstance.ward_clustering import WardClustering

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm

import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ds= buildReal("/data/dzeiberg/ClassPriorEstimation/rawDatasets/abalone.mat", 4)
ds = addTransformScores(ds)
ds.alphaHats, ds.curves = getBagAlphaHats(ds,numbootstraps=100)
ward = WardClustering(ds,randomPairing=True,numbootstraps=ds.alphaHats.shape[1])
ward.cluster()

In [ ]:
def moralize(row, clusterAssignment, alphaHatMat,numU):
    # cluster j was merged into cluster i at this iteration
    j = clusterAssignment[row-1,np.argmax(np.abs(clusterAssignment[row] - clusterAssignment[row - 1]))]
    i = clusterAssignment[row-1,np.argmin(np.abs(clusterAssignment[row] - clusterAssignment[row - 1]))]
    ai = alphaHatMat[row - 1, i].mean()
    aj = alphaHatMat[row - 1, j].mean()
    aij = alphaHatMat[row, i].mean()
    inI = np.where(clusterAssignment[row - 1] == i)[0]
    inJ = np.where(clusterAssignment[row - 1] == j)[0]
    ni = numU[inI].sum()
    nj = numU[inJ].sum()
    wi = ni / (ni + nj)
    ajHat = aij / (1 - wi) - wi / (1 - wi) * ai
    aiHat = (aij / wi) - ((1 - wi) / wi * aj)
    varI = alphaHatMat[row - 1,i].var()
    varJ = alphaHatMat[row - 1,j].var()
    if varI < varJ:
        #print(wi*ai+(1-wi)*aj,aij)
        print("update {j} from {aj:.3f} to {ajHat:.3f}".format(j=j,aj=alphaHatMat[row-1,j].mean(),ajHat=ajHat))
#         print(aij, wi*ai + (1-wi)*ajHat)
        print(ai, aiHat, aj, ajHat)
        alphaHatMat[row - 1,inJ] = ajHat
    else:
        print("update {i} from {ai:.3f} to {aiHat:.3f}".format(i=i,ai=alphaHatMat[row-1,i].mean(),aiHat=aiHat))
        alphaHatMat[row - 1,inI] = aiHat
    return alphaHatMat

In [ ]:
ahmat = np.copy(ward.alphaHatMat)
for row in range(ward.clusterAssignment.shape[0]-1,0,-1):
    ahmat = moralize(row, ward.clusterAssignment.astype(int), ahmat, ds.numU)

In [ ]:
ward.clusterAssignment

In [ ]:
plt.plot([np.mean(np.abs(ds.trueAlphas.flatten() - ahmat[i].mean(1))) for i in range(ahmat.shape[0]-1,-1,-1)])